In [1]:
"""
Notebook for training the embedding model for the flow around cylinder system.
=====
Distributed by: Notre Dame SCAI Lab (MIT Liscense)
- Associated publication:
url: https://arxiv.org/abs/2010.03957
doi: 
github: https://github.com/zabaras/transformer-physx
=====
"""
!nvidia-smi

zsh:1: command not found: nvidia-smi


# Environment Setup

In [2]:
!ls

data                             train_cylinder_enn.py
outputs                          train_cylinder_transformer.ipynb
train_cylinder_enn.ipynb         train_cylinder_transformer.py


Now lets download the training and validation data for the cylinder system. This will eventually be update to zenodo repo.

In [3]:
!mkdir data

mkdir: data: File exists


**WARNING: Training datafile is 1.3Gb! Validation is 0.35Gb!** These will be stored in your Google drive!

Because this is a big datafile, we will use gdown to instead of wget because of virus warning from google drive.

# Transformer-PhysX Cylinder System

Train the embedding model.
First import necessary modules from trphysx. 

In [4]:
import sys
import logging

import paddle
from paddle.optimizer.lr import ExponentialDecay

from trphysx.config.configuration_auto import AutoPhysConfig
from trphysx.embedding.embedding_auto import AutoEmbeddingModel
from trphysx.viz.viz_auto import AutoViz
from trphysx.embedding.training import *

logger = logging.getLogger(__name__)

Training arguments.

In [5]:
argv = []
argv = argv + ["--exp_name", "cylinder"]
argv = argv + ["--training_h5_file", "./data/cylinder_train.hdf5"]
argv = argv + ["--eval_h5_file", "./data/cylinder_valid.hdf5"]
argv = argv + ["--batch_size", "32"]
argv = argv + ["--block_size", "4"]
argv = argv + ["--n_train", "27"]
argv = argv + ["--n_eval", "6"]
argv = argv + ["--epochs", "100"]

In [6]:
args = EmbeddingParser().parse(args=argv)

# Setup logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)

if paddle.device.is_compiled_with_cuda():
    use_cuda = "cuda"
else:
    use_cuda = "cpu"
args.device = paddle.set_device(use_cuda)
logger.info("Torch device: {}".format(args.device))


INFO 2022-12-31 23:17:55,831 3764781669.py:15] Torch device: Place(cpu)


## Initializing Datasets and Models

Now we can use the auto classes to initialized the predefined configs, dataloaders and models. This may take a bit!

In [7]:
# Load transformer config file
config = AutoPhysConfig.load_config(args.exp_name)
data_handler = AutoDataHandler.load_data_handler(args.exp_name)
viz = AutoViz.load_viz(args.exp_name, plot_dir=args.plot_dir)

# Set up data-loaders
training_loader = data_handler.createTrainingLoader(
    args.training_h5_file,
    block_size=args.block_size,
    stride=args.stride,
    ndata=args.n_train,
    batch_size=args.batch_size,
)
testing_loader = data_handler.createTestingLoader(
    args.eval_h5_file, block_size=32, ndata=args.n_eval, batch_size=8
)

# Set up model
model = AutoEmbeddingModel.init_trainer(args.exp_name, config).to(args.device)
mu, std = data_handler.norm_params
# model.embedding_model.mu = mu.to(args.device)
# model.embedding_model.std = std.to(args.device)
if args.epoch_start > 1:
    model.load_model(args.ckpt_dir, args.epoch_start)


INFO 2022-12-31 23:17:55,840 enn_data_handler.py:300] Creating training loader
INFO 2022-12-31 23:17:59,092 enn_data_handler.py:389] Creating testing loader
WARNING 2022-12-31 23:17:59,411 enn_data_handler.py:425] Lower batch-size to 6
INFO 2022-12-31 23:17:59,428 embedding_cylinder.py:178] Number of embedding parameters: Tensor(shape=[1], dtype=int64, place=Place(cpu), stop_gradient=False,
       [262535])


Initialize optimizer and scheduler. Feel free to change if you want to experiment.

In [8]:
scheduler = ExponentialDecay(
    learning_rate=args.lr * 0.995 ** (args.epoch_start - 1), gamma=0.995
)
optimizer = paddle.optimizer.Adam(
    parameters=model.parameters(),
    learning_rate=scheduler,
    weight_decay=1e-8,
)


## Training the Embedding Model

This notebook only trains for 100 epochs for brevity, feel free to train longer. The test loss here is only the recovery loss MSE(x - decode(encode(x))) and does not reflect the quality of the Koopman dynamics.

In [ ]:
trainer = EmbeddingTrainer(model, args, (optimizer, scheduler), viz)
trainer.train(training_loader, testing_loader)

## Visualization of Results

Embed some test predictions of the embedding model, this is simply showing the prediction of the model of one Koopman step. I.e. x(t+1) = decoder(K*encoder(x(t))). Random time-steps are plotted.

In [ ]:
from IPython.display import Image, display

for epoch in [1, 25, 50, 75, 100]:
  print('Validation prediction for epoch: {:d}'.format(epoch))
  file_path = './outputs/embedding_cylinder/ntrain27_epochs100_batch32/predictions/embeddingPred0_{:d}.png'.format(epoch)
  display(Image(file_path, width=600, height=300))